마스크 분류

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
print(tf.keras.__version__)

2.2.4-tf


In [5]:
# 학습 데이터 경로
train_dir = 'https://github.com/Wildturkeyy/Deep_learning.git'

# 테스트 데이터 경로
test_dir = 'mask_dataset/test/'

In [4]:
# 이미지 가로 세로 
im_width = 224
im_height = 224

In [3]:
# 학습에 사용된 이미지를 읽을 객체

# 실행할 때마다 변형된 이미지를 리턴해서 이미지가 많은 것 같은 효과를 줌

train_datagen = ImageDataGenerator(
            rotation_range = 180, # 회전 최대 180도
            width_shift_range = 0.2, # 좌우 이동 최대 이미지 가로 사이즈 20%
            height_shift_range = 0.2, # 상하 이동 최대 이미지 세로 사이즈 20%
            horizontal_flip=True, # 좌우 반전 실행
            vertical_flip=True, # 상하 반전 실행
            rescale=1./225, # 이미지를 255로 나눠서 0~1 사이값으로 반환
            brightness_range=[0.5 , 1.2], # 이미지 밝기 조정
            # 완전 어두운 이미지 1, 원본밝기 0.5 : 원본 50%밝기, 1.2 : 원본보다 20%밝은 이미
            zoom_range = [0.8, 1.2] # 이미지 확대 0.8 원본의 80%확대, 1.2:원본의 120% 확대
)

In [6]:
train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(im_height, im_width),
            batch_size=64,
            class_mode='categorical',
            shuffle=True
)

OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'https://github.com/Wildturkeyy/Deep_learning.git'

In [7]:
# 검증에 사용될 이미지를 읽을 객체
# 테스트에서는 이미지를 변경할 필요가 없으므로 
# imagedatagenerator에 값을 주지 않음
# 이미지가 변형되지 않고 같은 이미지 리턴
# 이미지를 255로 나눠서 0~1 사이값으로 변환

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
test_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=(im_height, im_width),
            batch_size=64,
            class_mode='categorical',
            shuffle=False
)

Found 419 images belonging to 3 classes.


In [9]:
# 학습 데이터의 전체 이미지 개수를 리턴
train_generator.n

1659

In [10]:
# 한번에 리턴할 이미지의 개수
train_generator.batch_size

64

In [11]:
# 테스트 데이터의 전체 이미지 개수 리턴
test_generator.n

419

In [12]:
# train_generator.next()
# train_generator.batch_size에 설정된 이미지 128??개를 리턴해서 img에 대입
# 이미지의 label을 label에 대입
img, label = train_generator.next()

In [13]:
img.shape

(64, 224, 224, 3)

In [14]:
# one-hot encoding되어있음
label

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0

In [15]:
label.shape

(64, 3)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

In [17]:
# VGG16 구조를 갖는 model 생성
conv_layers = VGG16(
            weights='imagenet', # 이미지넷 대회에서 학습한 필터의 weight 그대로 사용
            include_top=False, # 맨 마지막의 100개로 분류하는 선형회귀 삭제
            input_shape=(im_height, im_width, 3) # 이미지의 im_width 가로 im_height 세로 3컬러
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [18]:
conv_layers.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [19]:
# 이미지넷 대회의 필터를 그대로 사용
for layer in conv_layers.layers:
    layer.trainable = False

    # 해당 레이어의 weight는 수정하지 않고 이미지넷 대회의 값 그대로 사용

In [20]:
# 입력값을 읽어서 예측할 Sequential 객체 생성
model = Sequential()

In [21]:
# VGG16 대입
model.add(conv_layers)

In [22]:
# 선형 회귀를 하기 위해서 합성곱 연산을 수행한 결과를 1차원 배열로 변환
model.add(Flatten())

In [23]:
# Dense: 선형회귀를 수행할 객체
# Dense(출력데이터의 칸의 수): 출력 데이터는 y_h1이고 칸의 수는 512이므로
# Dense(512)???
model.add(Dense(512, activation='relu'))

In [24]:
# 전체에서 임의의 50%만 학습
model.add(Dropout(0.5))

In [25]:
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

In [26]:
model.add(Dense(3, activation='softmax'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 7

In [28]:
# GradientDecent를 이용해서 w0 b0, w1, b1를 찾는 방법 설정
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['acc'])

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [35]:
# 학습시 가장 정확도가 높은 모델을 저장하도록 설정
cb_checkpoint = ModelCheckpoint(
                # filepath='./check/', # 학습 진행시 가장 정확도가 높은 모델 저장할 경로
                filepath='./check/saved_model.pb', # 학습 진행시 가장 정확도가 높은 모델 저장할 경로  '{epoch:02d}-{val_loss:.5f}.h5'
                monitor = 'val_acc', # 저장할 조건 val_acc(테스트 데이터의 정확도)가 가장 높은 모델 저 
                vervose = 1, # 함수의 진행 과정 출력
                save_best_only=True # 가장 정확도가 높은 모델 1개만 저장.
)               


In [36]:
# model.fit(train_generator, epoch=50) : train_generator를 이용해서 이미지를 리턴받음
# epoch를 진행할때 마다 증강된 이미지를 리턴
# 64개씩(batch_size) 50번 반복해서 학습

model.fit(
        train_generator,
        epochs=50,
        validation_data = test_generator,
        callbacks=[cb_checkpoint]
)

Epoch 1/50
26/26 [==============================] - 234s 9s/step - loss: 0.9773 - acc: 0.5063 - val_loss: 0.7742 - val_acc: 0.7088
Epoch 2/50
26/26 [==============================] - 238s 9s/step - loss: 0.9485 - acc: 0.5322 - val_loss: 0.7390 - val_acc: 0.7279
Epoch 3/50
26/26 [==============================] - 239s 9s/step - loss: 0.8888 - acc: 0.5817 - val_loss: 0.7117 - val_acc: 0.7351
Epoch 4/50
26/26 [==============================] - 243s 9s/step - loss: 0.8560 - acc: 0.5992 - val_loss: 0.6756 - val_acc: 0.7351
Epoch 5/50
26/26 [==============================] - 229s 9s/step - loss: 0.8618 - acc: 0.5955 - val_loss: 0.6544 - val_acc: 0.7446
Epoch 6/50
26/26 [==============================] - 227s 9s/step - loss: 0.8228 - acc: 0.6251 - val_loss: 0.6382 - val_acc: 0.7494
Epoch 7/50
26/26 [==============================] - 225s 9s/step - loss: 0.7780 - acc: 0.6552 - val_loss: 0.6149 - val_acc: 0.7422
Epoch 8/50
26/26 [==============================] - 225s 9s/step - loss: 0.7872 - a

KeyboardInterrupt: 

In [ ]:
import keras
# 저장한 모델을 읽어서 best_model 변수에 대입
best_model = keras.models.load_model('./check/saved')

In [ ]:
# best_model을 이용해 정확도 계산
best_model.evaluate(test_generator)

In [ ]:
# test_generator.next()
# test_generator.batch_size에 설정된 이미지 50개를 리턴해서 X_test에 대입
# 이미지의 label을  y_test에 대입
X_test, y_test = test_generator.next()

In [ ]:
print(X_test.shape)
X_test

In [ ]:
print(y_test.shape)
y_test

In [ ]:
# best_model.predict(X_test) :
# X_test가 어떤 이미지인지 분류
predict = best_model.predict(X_test)
predict

In [ ]:
# np.argmax(predict, 1) : predict의 최대값의 인덱스를 리턴
predict01 = np.argmax(predict, 1)
predict01

In [ ]:
# y_test 값도 최대값의 인덱스를 리턴
target = np.argmax(y_test, 1)

In [ ]:
predict02 = (predict01 == target)
predict02

In [ ]:
# True는 1로 False는 0으로 변환해서 합을 계산
np.sum(predict02)

In [ ]:
# 정확도 계산
acc = np.sum(predict02)/len(predict02)
acc

In [ ]:
# 레이블의 이름을 리턴
train_generator.class_indices
# train_generator.class_indices.keys()

In [ ]:
## 레이블의 이름을 리스트로 변환해서 custom_labels에 대입
custom_labels = list(train_generator.class_indices.keys())

In [ ]:
print(custom_labels[0])
print(custom_labels[1])
print(custom_labels[2])

In [ ]:
# 이미지를 출력할 객체
fig = plt.figrue(figsize=(30, 40))

for i in range(64):
    # 8줄 8칸으로 나누고 i+1번째 이미지를 그림
    subplot = fig.add_subplot(8, 8, i+1)

    #subplot.set_xticks([]) : 그래프의 x축을 설정
    # 데이터가 없으므로 x축에 아무 데이터도 출력 안됨
    subplot.set_xticks([])
    subplot.set_yticks([])

    # 예측값(숫자)을 predict_num에 대입
    predict_num = predict01[i]
    # 예측값에 해당하는 문자를 리턴
    predict_str = custom_labels[predict_num]

    # 실제 값 (숫자) 리턴
    target_num = target[i]
    # 실제 값에 해당하는 문자 리턴